In [1]:
import carla
import random
import time
import numpy as np
import struct
import os

In [2]:
# Connect to Carla server
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
tm = client.get_trafficmanager()
tm.set_synchronous_mode(True)

# Load a town (optional)
#town_name = 'Town03'
#world = client.load_world(town_name)

# Set the simulation FPS
desired_fps = 10  # Set your desired FPS here
settings = world.get_settings()
settings.fixed_delta_seconds = 1.0 / desired_fps
settings.synchronous_mode = True  # Enable synchronous mod
world.apply_settings(settings)



4904

In [3]:
#Spawn random vehicles with autopilot
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')

spawn_points = world.get_map().get_spawn_points()
random.shuffle(spawn_points)

# Limit to 50 cars or available spawn points
num_cars = min(50, len(spawn_points))

vehicles = []

for spawn_point in spawn_points[:num_cars]:
    for _ in range(10):  # Try up to 10 times to spawn the vehicle
        vehicle = world.try_spawn_actor(random.choice(vehicle_bp), spawn_point)
        if vehicle:
            vehicle.set_autopilot(True)
            vehicles.append(vehicle)
            break  # Exit the retry loop if successful

In [4]:
#Spawn your car with autopilot and LiDAR sensor
ego_vehicle_bp = blueprint_library.find('vehicle.audi.a2')
ego_vehicle_spawn_point = spawn_points[num_cars]

ego_vehicle = world.spawn_actor(ego_vehicle_bp, ego_vehicle_spawn_point)
ego_vehicle.set_autopilot(True)

lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100')  # Increase range if needed
lidar_bp.set_attribute('rotation_frequency', str(desired_fps))
lidar_bp.set_attribute('channels', '64')  # Increase number of channels for better vertical resolution
lidar_bp.set_attribute('points_per_second', '1000000')  # Increase points per second for denser point cloud
lidar_bp.set_attribute('upper_fov', '10')  # Adjust upper field of view
lidar_bp.set_attribute('lower_fov', '-30')  # Adjust lower field of view
lidar_bp.set_attribute('horizontal_fov', '360')  # Ensure full 360 degree horizontal field of view

lidar_location = carla.Location(0, 0, 2.5)
lidar_rotation = carla.Rotation(0, 0, 0)
lidar_transform = carla.Transform(lidar_location, lidar_rotation)
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=ego_vehicle)

In [5]:
# Ensure the folder 'lidar_data' exists
if not os.path.exists('lidar_data'):
    os.makedirs('lidar_data')

#Function to process LiDAR data and save it as a binary file
def save_lidar_data(data):
    global frame
    points = np.frombuffer(data.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (int(points.shape[0] / 4), 4))
    
    file_name = f'lidar_data/lidar_frame_{frame}.bin'
    with open(file_name, 'wb') as f:
        for point in points:
            f.write(struct.pack('ffff', *point))
            
    frame += 1  # Increment the frame counter

In [6]:
#Attach callback to LiDAR sensor and start data collection
frame = 0
lidar.listen(lambda data: save_lidar_data(data))

In [7]:
# Tick the server to gather 1000 frames of data
for _ in range(1000):
    world.tick()

In [8]:
# Always disable sync mode before the script ends to prevent the server blocking whilst waiting for a tick
settings.synchronous_mode = False
settings.fixed_delta_seconds = None
world.apply_settings(settings)
tm.set_synchronous_mode(False)

#Cleanup
lidar.stop()
lidar.destroy()
ego_vehicle.destroy()

True

In [ ]:
for vehicle in vehicles:
    vehicle.destroy()

In [2]:
# Get all actors in the world
actors = world.get_actors()

# Filter for vehicle actors and print their blueprints
vehicle_actors = actors.filter('vehicle.*')

print(vehicle_actors)

NameError: name 'world' is not defined

In [1]:
vehicles

NameError: name 'vehicles' is not defined